# Example notebook for generating random models using package generateModel

Models included are: 
*Erdos Renyi, 
*Stochastic Block Model, 
*Distance Dependant (2nd order), 
*Distance Dependant (3rd order)

Output of all functions is a matrix in coo format stored as a dictionary with two entries ['row'] and ['col']

Install with "pip install .", requires cmake>=2.8.12 and gcc>=(not sure TODO)

In [1]:
import generateModel as gm
import numpy as np
import pandas

## Run on random 100 vertex examples

In [2]:
#Erdos-Renyi model:
#Input: n, p, threads
#    n = number of vertices (int)
#    p = edge probability (double)
#    threads = number of threads to use (int)
#Example
n = 100
p = 0.5
threads = 8
A = gm.ER(n,p,threads)

In [3]:
#Stochastic Block Model model:
#Input: n, pathways, mtypes, threads
#    n = number of vertices (int)
#    M = M[i][j] entry is probability of edge between mtype i and mtype j (numpy array, shape=(m,m), dtype=float64), where m is number of mtypes
#    mtypes = i'th entry is mtype of vertex i (numpy array, shape=(n,), dtype=uint8)
#    threads = number of threads to use (int)
#Example
n = 100
m = 10
pathways = np.random.rand(m,m)
mtypes = np.random.randint(m,size=(n,))
threads = 8

B = gm.SBM(n, pathways, mtypes, threads)

In [4]:
#Distance Dependant 2nd Order:
#Input: n, a, b, xyz, threads
#    n = number of vertices (int)
#    a = coefficient of probability function (double)
#    b = coefficient of probability function (double)
#    xyz = the coordinates of the vertices, (numpy array, shape=(n,3), dtype=float64)
#    threads = number of threads to use (int)
#Example
n = 100
a = 0.085
b = -0.005
xyz = np.random.rand(n,3)
threads = 8

C = gm.DD2(n,a,b,xyz,threads)

In [5]:
#Distance Dependant 3rd Order:
#Input: n, a, b, xyz, threads
#    n = number of vertices (int)
#    a1 = coefficient of probability function for dz<0 (double)
#    b1 = coefficient of probability function for dz<0 (double)
#    a2 = coefficient of probability function for dz>0 (double)
#    b2 = coefficient of probability function for dz>0 (double)
#    xyz = the coordinates of the vertices, (numpy array, shape=(n,3), dtype=float64)
#    depths = i'th entry is depth of vertex i (numpy array, shape=(n,), dtype=float64)
#    threads = number of threads to use (int)
#Example
n = 100
a1 = 0.089
b1 = -0.003
a2 = 0.091
b2 = -0.008
xyz = np.random.rand(n,3)
depths = np.random.rand(n)
threads = 8

D = gm.DD3(n,a1,b1,a2,b2,xyz,depths,threads)

## Run on full SSCX
Below are examples of how to run the above models on the full SSCX21_BioM circuit

In [ ]:
#load data
neurons_address = "/gpfs/bbp.cscs.ch/project/proj102/matrices/SSCX21_BioM/neuron_info.feather"
neurons = pandas.read_feather(neurons_address)

n = len(neurons)
num_edges = len(np.load('local_connectivityindices.npy'))
p = num_edges/(n*(n-1))

xyz = np.array(neurons[['x','y','z']])
depths = np.array(neurons['depth'])

#convert mtypes from string to integer
num_mtype = len(neurons['mtype'].cat.categories)
mtype_lookup = {neurons['mtype'].cat.categories[i]:i for i in range(len(num_mtype))}
mtypes = [mtype_lookup[neurons['mtype'].iloc[i]] for i in range(n)]

pathways = np.random.rand(num_mtype,num_mtype) #Using random pathway probability for now

a = 0.085
b = -0.005
a1 = 0.089
b1 = -0.003
a2 = 0.091
b2 = -0.008
threads = 40

In [ ]:
#Run the models
SSCX_ER = gm.ER(n,p,threads)
SSCX_SBM = gm.SBM(n, pathways, mtypes, threads)
SSCX_DD2 = gm.DD2(n,a,b,xyz,threads)
SSCX_DD3 = gm.DD3(n,a1,b1,a2,b2,xyz,depths,threads)